In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset


In [47]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(LSTMModel, self).__init__()
        
        self.hidden_sizes = hidden_sizes
        
        self.lstm = nn.LSTM(input_size, hidden_sizes[0], num_layers=3, batch_first=True)
        self.hidden_layers = nn.ModuleList(
            [ nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes)-1)]
        )
        self.fc = nn.Linear(hidden_sizes[-1], output_size)
        
    def forward(self, x):
        batch_size = x.size(0)
        seq_len = x.size(1)
        
        h0 = torch.zeros(3, batch_size, self.hidden_sizes[0]).to(x.device)
        c0 = torch.zeros(3, batch_size, self.hidden_sizes[0]).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        for layer in self.hidden_layers:
            out = layer(out)
            out = torch.relu(out)
        
        out = self.fc(out[:, -1, :])  # Mengambil output terakhir dari LSTM
        
        return out
# Membangun model
input_size = 7
hidden_sizes = [3, 4, 5]
output_size = 1

model = LSTMModel(input_size, hidden_sizes, output_size)

In [11]:
model

LSTMModel(
  (lstm): LSTM(7, 3, num_layers=3, batch_first=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): Linear(in_features=4, out_features=5, bias=True)
  )
  (fc): Linear(in_features=5, out_features=1, bias=True)
)

In [44]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            
            inputs = inputs.squeeze(0)
        
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
        
            optimizer.step()
        
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1} / {num_epochs}, Loss = {epoch_loss:.4f}")

In [20]:
def test(model, test_loader):
    model.eval()
    predictions = []
    targets = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            
            predictions.extend(output.squeeze().tolist())
            targets.extend(labels.squeeze().tolist())
    
    return predictions, targets

In [40]:
class ForexDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        sample = self.data[index]
        inputs = torch.Tensor(sample[:-1])
        label = torch.Tensor([sample[-1]])
        return inputs, label


# Contoh penggunaan

# Data latih acak
train_data = [
    [1.2, 2.3, 3.4, 4.5, 5.6, 6.7],
    [2.1, 3.2, 4.3, 5.4, 6.5, 7.6],
    [3.4, 4.5, 5.6, 6.7, 7.8, 8.9],
    # ... tambahkan data latih lainnya
]

# Data uji acak
test_data = [
    [4.5, 5.6, 6.7, 7.8, 8.9, 9.0],
    [5.6, 6.7, 7.8, 8.9, 9.0, 9.1],
    # ... tambahkan data uji lainnya
]

# Inisialisasi dataset dan loader
train_dataset = ForexDataset(train_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = ForexDataset(test_data)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [41]:
# implementations

In [42]:
criterion = nn.MSELoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [48]:
num_epoch = 10
train(model, train_loader, criterion, optimizer, num_epoch)

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors